import packages

In [1]:
import torch
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from torch.autograd import Variable

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from importlib import import_module
import sys, os
opj = os.path.join
ope = os.path.exists

In [3]:
this_dir = os.getcwd()
lib_path = opj(this_dir, 'bestfitting/protein_clean/src')
if lib_path not in sys.path:
    sys.path.insert(0, lib_path)

In [4]:
import train_cls_net # import Protein class
from net import _init_paths
from config.config import * # set directory paths (DATA_DIR, RESULT_DIR etc)
from dataset import protein_dataset # import ProteinDataset class
from utils.augment_util import * # import augmentation functions
from net.loss_funcs.kaggle_metric import prob_to_result # import prob_to_result
from net.loss_funcs.kaggle_metric import get_probs_f1_score # import get_probs_f1_score

run on ip-172-31-62-205


### Check the number of images and labels 

train data

In [5]:
img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/tif/train/tifs'
img_resize_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/train/images_1536'

# load image file names
img = np.sort(os.listdir(img_dir))
img_resize = np.sort(os.listdir(img_resize_dir))

# load label name
fname = opj(DATA_DIR, 'split/train_31072.csv')
df = pd.read_csv(fname)

In [9]:
# print file sizes
print(fname,':', df.values.shape)
print(int(img.shape[0]/4))
print(int(img_resize.shape[0]/4))

/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/train_31072.csv : (31072, 31)
31072
31072


external data

In [20]:
external_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/externel/HPAv18/jpg'
external_img_resize_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/train/external_v18_1536'

# load file names
external_img = np.sort(os.listdir(external_img_dir))
external_resize = np.sort(os.listdir(external_img_resize_dir))

# load label name
fname = opj(DATA_DIR, 'split/external_74606.csv')
df = pd.read_csv(fname)

In [21]:
# print file sizes
print(fname,':', df.values.shape)
print(int(external_img.shape[0]/4))
print(int(external_resize.shape[0]/4))

/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/external_74606.csv : (74606, 33)
74606
74606


random_folds5

In [22]:
split_alias = 'random_folds5'
num = []
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_train_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape)
    num.append(df.values.shape[0])
    
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_valid_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape, ';sum:', num[fold] + df.values.shape[0])    

/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_train_cv0.csv : (24922, 31)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_train_cv1.csv : (24801, 31)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_train_cv2.csv : (24822, 31)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_train_cv3.csv : (24893, 31)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_train_cv4.csv : (24850, 31)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_valid_cv0.csv : (6150, 31) ;sum: 31072
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_valid_cv1.csv : (6271, 31) ;sum: 31072
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_valid_cv2.csv : (6250, 31) ;sum: 31072
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/d

random_ext_folds5

In [28]:
split_alias = 'random_ext_folds5'
num = []
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_train_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape)
    num.append(df.values.shape[0])
    
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_valid_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape, 'sum:', num[fold] + df.values.shape[0])    

/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_train_cv0.csv : (84587, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_train_cv1.csv : (84563, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_train_cv2.csv : (84485, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_train_cv3.csv : (84588, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_train_cv4.csv : (84489, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_valid_cv0.csv : (21091, 33) sum: 105678
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_valid_cv1.csv : (21115, 33) sum: 105678
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_valid_cv2.csv : (21193, 33) sum: 105678
/home/ubuntu/HPA/hpa

random_ext_noleak_clean_folds5

In [27]:
split_alias = 'random_ext_noleak_clean_folds5'
num = []
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_train_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape)
    num.append(df.values.shape[0])
    
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_valid_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape, 'sum:', num[fold] + df.values.shape[0])    

/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_train_cv0.csv : (82747, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_train_cv1.csv : (82709, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_train_cv2.csv : (82643, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_train_cv3.csv : (82765, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_train_cv4.csv : (82660, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_valid_cv0.csv : (19984, 33) sum: 102731
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_valid_cv1.csv : (20062, 33) sum: 102771
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/

In [ ]:
# header = 'external'
# fname = opj(DATA_DIR, 'meta/%s_meta.csv'%(header))
# df = pd.read_csv(fname)
# print(fname,':', df.values.shape)

# # image ids for external dataset 
# external_id = df['Id'].values
# external_id = np.sort(external_id)

# # image directory
# external_orig_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/externel/HPAv18/jpg'
# external_orig2_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/externel/HPAv18_2/jpg'
# external_resize_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/train/external_v18_1536'

# # load file names
# external_orig = np.sort(os.listdir(external_orig_img_dir))
# external_orig2 = np.sort(os.listdir(external_orig2_img_dir))
# external_resize = np.sort(os.listdir(external_resize_img_dir))

# # find images which are not in external_reize 
# idx = np.argwhere(~np.isin(external_orig, external_resize)).ravel()
# wrong_fnames = external_orig[idx]
# print(len(wrong_fnames))

# # check if wrong files in external_orig are correct in external_orig2
# num = len(wrong_fnames)
# none_list = np.empty(num, dtype=bool)
# none_checking = np.empty(num, dtype=bool)

# for i in range(num):
#     fname = wrong_fnames[i]
#     color = fname.replace('.jpg','').split('_')[-1]
#     im = cv2.imread(opj(external_orig_img_dir, fname))
#     if im is None:
#         none_list[i] = True
#     else:
#         none_list[i] = False
        
#     im_checking = cv2.imread(opj(external_orig2_img_dir, fname))
#     if im_checking is None:
#         none_checking[i] = True
#     else:
#         none_checking[i] = False
# print(none_checking)

# import shutil
# # remove wrong files in external_orig and move from external_orig2
# for i in range(num):
#     fname = wrong_fnames[i]
#     os.remove(opj(external_orig_img_dir, fname))
#     shutil.move(opj(external_orig2_img_dir, fname), opj(external_orig_img_dir, fname))

#     ### rerun resize code after moving files ###
